<a href="https://colab.research.google.com/github/Soot3/AutoChek-Analysis/blob/main/AutoChek_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd

In [2]:
# getting the links to the cars

urls = []

for i in range(1,16):
  website = "https://autochek.africa/ng/cars-for-sale?country=ng&page_number={}&page_size=100".format(i)
  page = requests.get(website)
  soup = bs4(page.text, 'html.parser')
  for item in soup.findAll('a', href=True, class_="jsx-3456531073 car-item hover:tw-shadow-md"):
    link = item['href']
    link = link.replace('/undefined/', 'https://autochek.africa/ng/')
    urls.append(link)

In [3]:
urls.remove("https://autochek.africa/ng/car/nissan-trade-open-roof-truck-ref-7wvinthJf")
print(len(urls))
urls[-1]

1466


'https://autochek.africa/ng/car/toyota-sienna-ref-XOV8buAnU'

In [4]:
# Getting car information
title = []
#manufacturer = []
#model = []
#year = []
prices = []
odometer = []
location = []
used_where = []
engine = []
transmission = []
paint = []
fuel = []
for i in urls:
  page = requests.get(i)
  soup = bs4(page.text, "html.parser")
  # The headline is formatted like this "Toyota Camry 2012"
  headline = soup.get_text().split(',')[0]
  title.append(headline)
  #manufacturer.append(headline.split(' ')[0])
  #model.append(headline.split(' ')[1])
  #year.append(headline.split(' ')[2])
  for j in soup.findAll('div', class_='price'):
    price = j.getText()
    prices.append(price)
  eng = soup.findAll('span', class_='value text-capitalize')[0].string.extract()
  engine.append(eng)
  trans = soup.findAll('span', class_='value text-capitalize')[1].string.extract()
  transmission.append(trans)
  fu = soup.findAll('span', class_='value text-capitalize')[2].string.extract()
  fuel.append(fu)
  pa = soup.findAll('span', class_='value text-capitalize')[4].string.extract()
  paint.append(pa)
  odd = soup.find('div', class_='details-item-container tw-mt-4').findAll('span', class_='text')[0].getText()
  odometer.append(odd)
  loc = soup.find('div', class_='details-item-container tw-mt-4').findAll('span', class_='text')[1].getText()
  location.append(loc)
  isexported = soup.find('div', class_='details-item-container tw-mt-4').findAll('span', class_='text')[2].getText()
  used_where.append(isexported)



In [5]:
for i in [title, prices, odometer, location, used_where, engine, transmission, paint, fuel]:
  print(len(i))


1466
1466
1466
1466
1466
1466
1466
1466
1466


In [6]:
final_dict = {"title":title, "odometer": odometer, "location": location, "isimported": used_where, "engine": engine, "transmission": transmission, "fuel": fuel, "paint": paint, "price": prices}

In [7]:
df = pd.DataFrame(final_dict)

In [8]:
df.sample(10)

,title,odometer,location,isimported,engine,transmission,fuel,paint,price
1105,Honda CR-V 2018 Price,"27,079 km",Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Gray,"15,215,000"
1050,Lexus RX 400 2006 Price,"197,563 km",Lagos,Foreign Used,6-cylinder(I6),automatic,petrol,Brown,"3,975,000"
74,Toyota Coaster 2014 Price,0 km,Lagos,Locally used,4-cylinder(I4),manual,diesel,yellow,"16,690,000"
117,Honda Crosstour 2010 Price,"156,135 km",Adamawa,Locally used,6-cylinder(V6),automatic,petrol,Black,"2,715,000"
1060,Honda Accord 1996 Price,"247,731 km",Lagos,Locally used,4-cylinder(I4),automatic,petrol,Black/Red,"455,000"
1405,Lexus RX 350 2012 Price,"56,558 km",Lagos,Foreign Used,6-cylinder(I6),automatic,petrol,Grey,"9,015,000"
998,Toyota Sienna 2004 Price,"198,763 km",Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Silver,"2,760,000"
1260,Land Rover Range Rover Sport 2014 Price,"49,293 km",Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,White,"23,515,000"
568,Mercedes-Benz E 350 2014 Price,"49,422 km",Lagos,Foreign Used,6-cylinder(I6),automatic,petrol,Black,"11,015,000"
436,Honda Civic 2016 Price,"60,242 km",Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Black,"8,815,000"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466 entries, 0 to 1465
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1466 non-null   object
 1   odometer      1466 non-null   object
 2   location      1466 non-null   object
 3   isimported    1466 non-null   object
 4   engine        1466 non-null   object
 5   transmission  1466 non-null   object
 6   fuel          1466 non-null   object
 7   paint         1466 non-null   object
 8   price         1466 non-null   object
dtypes: object(9)
memory usage: 103.2+ KB


In [10]:
df.describe( include="all")

,title,odometer,location,isimported,engine,transmission,fuel,paint,price
count,1466,1466,1466,1466,1466,1466,1466,1466,1466
unique,682,1403,15,3,9,2,2,90,478
top,Toyota Camry 2011 Price,0 km,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Black,"3,315,000"
freq,25,45,1145,1065,900,1379,1386,410,24


In [11]:
df.price = df.price.str.replace(',', '')
df.odometer=df.odometer.str.replace(',','').str.replace(' km', '')

In [12]:
df.title = df.title.str.replace(' Price','')

In [13]:
df['year'] = df['title'].str.extract(r'(\d{4})') 
df['title'] = df['title'].str.replace(r' (\d{4})','')

In [14]:
df.title.value_counts().head(10)

Toyota Camry         181
Toyota Corolla       160
Toyota Highlander     80
Toyota Sienna         78
Toyota RAV 4          59
Honda Accord          54
Lexus RX 350          48
Lexus ES 350          32
Lexus GX 460          26
Toyota Venza          24
Name: title, dtype: int64

In [15]:
df[df['year'].isna()]

,title,odometer,location,isimported,engine,transmission,fuel,paint,price,year
71,DAF 95XF 0,0,Lagos,Locally used,8-cylinder(V8),manual,diesel,white-yellow,14965000,NaN
73,Iveco STRALIS 0,0,Lagos,Locally used,6-cylinder(I6),manual,diesel,white,8640000,NaN
82,Fiat Ducato 0,0,Lagos,Locally used,4-cylinder(I4),manual,diesel,white,3925000,NaN
85,Iveco 260-25ah-ahb 0,0,Lagos,Locally used,6-cylinder(I6),manual,diesel,white,9215000,NaN
87,Scania P113 0,0,Lagos,Locally used,6-cylinder(I6),manual,diesel,white,6340000,NaN
457,Scania TRACTOR HEAD 0,0,Lagos,Locally used,6-cylinder(V6),manual,diesel,white,8015000,NaN
461,Fiat Ducato 0,302235,Lagos,Locally used,4-cylinder(I4),manual,diesel,black,1715000,NaN
462,Iveco HIAB WITH CRANE 0,18909,Lagos,Locally used,6-cylinder(V6),manual,diesel,orange,4515000,NaN
463,Mack TRAILER 0,0,Lagos,Locally used,6-cylinder(V6),manual,diesel,brown,3515000,NaN
479,Scania TIPPING TRUCK 0,822263,Lagos,Locally used,6-cylinder(V6),manual,diesel,red,10015000,NaN


In [16]:
df.dropna(inplace=True)

In [17]:
df.price = df.price.astype(int)
df.odometer = df.odometer.astype(int)
df.year = df.year.astype(int)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1451 entries, 0 to 1465
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1451 non-null   object
 1   odometer      1451 non-null   int64 
 2   location      1451 non-null   object
 3   isimported    1451 non-null   object
 4   engine        1451 non-null   object
 5   transmission  1451 non-null   object
 6   fuel          1451 non-null   object
 7   paint         1451 non-null   object
 8   price         1451 non-null   int64 
 9   year          1451 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 124.7+ KB


In [19]:
df.sample(10)

,title,odometer,location,isimported,engine,transmission,fuel,paint,price,year
273,Toyota Highlander,30951,Lagos State,Foreign Used,6-cylinder(V6),automatic,petrol,Silver,16015000,2017
529,Toyota Camry,31600,Abuja,Foreign Used,4-cylinder(I4),automatic,petrol,Silver,3615000,2011
332,Volkswagen 17.22,142623,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Black,3165000,2010
930,Lexus RX 350,176862,Abuja,Foreign Used,6-cylinder(V6),automatic,petrol,Silver,9015000,2013
786,Toyota Corolla,113085,Lagos,Foreign Used,6-cylinder(I6),automatic,petrol,Gray,5790000,2014
1024,Lexus GX 460,44089,Lagos,Foreign Used,8-cylinder(V8),automatic,petrol,White,17395000,2014
686,Toyota Camry,102904,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Silver,3645000,2011
1066,Lexus RX 350,2584,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Black,24015000,2018
859,Toyota Matrix,65483,Abuja,Foreign Used,4-cylinder(I4),automatic,petrol,Blue,3315000,2009
387,Lexus RX 350,72903,Lagos,Foreign Used,6-cylinder(I6),automatic,petrol,Grey,9930000,2015


In [20]:
df.to_csv('autochek_scrape.csv', index=False, encoding='utf-8')